In [ ]:
Google_Drive = True #@param {type:"boolean"}

Checkpoint_Google_Save = True #@param {type:"boolean"}

Ngrok_Key = '' #@param {type:"string"}
Localtunnel = False #@param {type:"boolean"}

Google_Drive_Dir = 'webui' #@param {type:"string"}
Checkpoint_Url = 'https://civitai.com/api/download/models/125411?type=Model&format=SafeTensor&size=pruned&fp=fp16' #@param {type:"string"}
Checkpoint_Filename = 'realisticVision_v50.safetensors' #@param {type:"string"}

ControlNet = True #@param {type:"boolean"}
DDetailer = True #@param {type:"boolean"}
ADetailer = True #@param {type:"boolean"}
SegmentAnything = True #@param {type:"boolean"}

Workspace = 'ui'
NotebookVersion = 'stable'

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')

Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion/google'

# https://github.com/camenduru 참고
# 메모리 Fix
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2

%cd /content
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/install_{NotebookVersion}.sh -O /content/install.sh
!bash /content/install.sh {Workspace} {ControlNet} {DDetailer} {ADetailer} {SegmentAnything}

# 구글드라이브 연결
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/link_google_drive.sh -O /content/link_google_drive.sh
!bash /content/link_google_drive.sh {Workspace} {Google_Drive} {Google_Drive_Dir}

# 커스터마이징
%cd /content/{Workspace}
!bash /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/install.sh {Workspace}
if Google_Drive:
  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/config.json /content/{Workspace}/config.json
  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/ui-config.json /content/{Workspace}/ui-config.json
  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/styles.csv /content/{Workspace}/styles.csv

# checkpoint select
!sed -i -e 's/"sd_model_checkpoint": "",/"sd_model_checkpoint": "{Checkpoint_Filename}",/g' /content/{Workspace}/config.json

# upscaler
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/{Workspace}/models/ESRGAN -o 4x-UltraSharp.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/{Workspace}/models/VAE -o kl-f8-anime2.ckpt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o add_detail.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/63006?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o LowRA.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16" -d /content/{Workspace}/models/Lora -o blindbox_V1Mix.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/91700?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o koreaface15.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o add_detail.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/{Workspace}/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/{Workspace}/embeddings -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d /content/{Workspace}/embeddings -o bad-hands-5.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o bad_prompt_version2.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/5119?type=Model&format=SafeTensor" -d /content/{Workspace}/embeddings/ -o pureerosface_v1 .pt

# depthmask
!git clone https://github.com/Extraltodeus/depthmap2mask.git /content/{Workspace}/extensions/depthmap2mask

if Localtunnel:
  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")
    
    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

if Ngrok_Key:
  !python launch.py --xformers --no-half-vae --theme dark --ngrok {Ngrok_Key}
else:
  !python launch.py --share --xformers --no-half-vae --theme dark --gradio-queue --api
